In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import akshare as ak
from datetime import datetime, timedelta
from IPython.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  # 也可以设置列宽

In [ ]:

# 导入数据
# df = pd.read_csv('data.csv')

def fetch_data(symbol: str, start_date: str = None, end_date: str = None, adjust: str = ""):
    start_date = (datetime.now() - timedelta(days=500)
                  ).strftime('%Y%m%d') if not start_date else start_date
    end_date = datetime.now().strftime('%Y%m%d') if not end_date else end_date
    # 日期    开盘    收盘    最高    最低     成交量           成交额    振幅   涨跌幅   涨跌额   换手率
    try:
        df = ak.stock_zh_a_hist(
            symbol=symbol, start_date=start_date, end_date=end_date, adjust='')
    except Exception as e:
        print(f'ak.stock_zh_a_hist调用出错。{e}')
    return df


symbol = "300031"  
df = fetch_data(symbol=symbol)
df.rename(columns={"日期":"date","开盘":"open","最高":"high","最低":"low","收盘":"close","成交量":"volume"},inplace=True)
df=df[["date","open","high","low","close","volume"]]



In [ ]:
import time
def generate_data_chunks(df, period=2):
    index = 0
    while index < len(df):
        yield df.head(index + 1)
        time.sleep(period)
        index += 1

In [ ]:
import numpy as np
import pandas as pd
import random
import time

def simulate_tohlcv(frequencies, amplitudes, period=2, initial_price=100.00):
    t = 0  # 时间变量，用于正弦波的计算
    while True:
        price_waves = [amp * np.sin(2 * np.pi * freq * t) for freq, amp in zip(frequencies, amplitudes)]
        open_price = round(initial_price + sum(price_waves), 2)

        high_price = low_price = open_price
        # 模拟周期内的价格变动
        prices = [open_price]
        for _ in range(period - 1):  # 模拟period秒内的价格变动
            change = round(np.random.uniform(-0.01, 0.01) * high_price, 2)
            new_price = max(95.00, min(105.00, prices[-1] + change))
            prices.append(new_price)

        high_price = round(max(prices), 2)
        low_price = round(min(prices), 2)
        close_price = round(prices[-1], 2)

        current_time = pd.Timestamp.now().strftime('%H:%M:%S')
        volume = random.randint(500, 5000)  # 随机生成成交量
        yield {
            'time': current_time,
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'volume': volume
        }
        t += 1 / (60 / period)  # 根据周期调整时间变量的增加量
        time.sleep(period - 0.01)  # 等待直到下一个周期开始，减去0.01秒以补偿计算时间


In [ ]:
import pandas_ta as ta
import pandas as pd

class CMF_EMA_SignalGenerator:
    def __init__(self, short_window=3, long_window=10, cmf_window=3):
        self.short_window = short_window
        self.long_window = long_window
        self.cmf_window = cmf_window

    def update_with_new_data(self, df):
        """
        使用新的实时数据更新DataFrame和信号。
        
        参数:
        - new_data: 包含最新数据的新DataFrame行，应包含'high', 'low', 'close', 'volume'列。
        """
        if len(df) < self.long_window: return
        # 将新数据添加到DataFrame
        self.df = df.copy()
        
        # 重新计算信号
        self.generate_signals()
        
        columns=["close", "cmf","trend_up","trend_down","upward_trend","downward_trend", "signal"]
        columns=["close", "cmf","trend_up","trend_down","upward_trend","downward_trend", "signal"]
        # columns=["open", "high","low","close","volume","cmf", "signal"]
        # print(self.df[columns].tail(5))        
        # 返回最新的信号
        if self.df['signal'].iloc[-1] >0 or self.df['signal'].iloc[-1] < 0:
            columns=["close", "ema_short","ema_long","cmf", "trend_up","trend_down","upward_trend","downward_trend", "signal"]
            display(HTML(self.df[columns].tail(5).to_html(index=True)))
        return self.df['signal'].iloc[-1]

    def generate_signals(self):
        # 计算CMF和EMA
        self.df['cmf'] = ta.cmf(
            high=self.df['high'],
            low=self.df['low'],
            close=self.df['close'],
            volume=self.df['volume'],
            length= self.cmf_window
        )
        self.df['ema_short'] = round(ta.ema(self.df['close'], self.short_window),2)
        self.df['ema_long'] = round(ta.ema(self.df['close'], self.long_window),2)
        
                               
        # 检查 CMF 是否持续 n 个周期向上或向下
        self.df['trend_up'] = self.df['cmf'] > self.df['cmf'].shift(1)
        self.df['trend_down'] = self.df['cmf'] < self.df['cmf'].shift(1)
        n = self.cmf_window  # 持续周期的数量
        self.df['upward_trend'] = self.df['trend_up'].shift(1).rolling(window=n).apply(lambda x: all(x), raw=True)
        self.df['downward_trend'] = self.df['trend_down'].shift(1).rolling(window=n).apply(lambda x: all(x), raw=True)                                     
        self.df['upward_trend'] = self.df['upward_trend'].astype(bool)
        self.df['downward_trend'] = self.df['downward_trend'].astype(bool)
        # 交易信号逻辑
        self.df['buy_signal'] = self.df['upward_trend'] & (
            self.df['ema_short'] > self.df['ema_long']
        )
        self.df['sell_signal'] =  self.df['downward_trend'] & (
            self.df['ema_short'] < self.df['ema_long']
        )
        # self.df['buy_signal'] = ((self.df['cmf'] > 0) & (self.df['cmf'].shift(1) <= 0)) & (
        #     self.df['ema_short'] > self.df['ema_long']
        # )
        # self.df['sell_signal'] = ((self.df['cmf'] < 0) & (self.df['cmf'].shift(1) >= 0)) | (
        #     self.df['ema_short'] < self.df['ema_long']
        # )

        # 将信号标记为1或-1
        self.df['signal'] = 0
        self.df.loc[self.df['buy_signal'], 'signal'] = 1
        self.df.loc[self.df['sell_signal'], 'signal'] = -1



In [ ]:

# 使用示例
if __name__ == "__main__":
    frequencies = [0.1, 0.05, 0.02]  # 定义三个正弦波的频率
    amplitudes = [2, 1.5, 1]  # 定义三个正弦波的振幅
    simulator = simulate_tohlcv(frequencies, amplitudes)
    data_points = []

    signal_generator = CMF_EMA_SignalGenerator()
    while(True):  
        data_point = next(simulator)
        data_points.append(data_point)
        df = pd.DataFrame(data_points)  # 将数据点转换为DataFrame
        # print(df)  # 实时输出DataFrame
        # 更新信号生成器并获取最新信号
        latest_signal = signal_generator.update_with_new_data(df)
        if latest_signal and latest_signal !=0:
            print("Latest signal:", latest_signal)            


In [ ]:
if __name__ == "__main__":
    generator = generate_data_chunks(df)
    current_chunk = None
    signal_generator = CMF_EMA_SignalGenerator()
    while current_chunk is None or len(current_chunk) < len(df):
        current_chunk = next(generator)
        # print(current_chunk)
        latest_signal = signal_generator.update_with_new_data(current_chunk)
        if latest_signal and latest_signal !=0:
            print("Latest signal:", latest_signal)          
